# ___Imports___

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
import plotly.express as px
from geopy.geocoders import Nominatim
import geopy as gp
from datetime import datetime
from glob import glob

# ___Load Data___

In [2]:
# load data for nasa
data = pd.read_csv('../data/ALL_DATA.csv')
print(data.columns)

data.head()
np.shape(data)

Index(['id', 'class', 'mass', 'fall', 'year', 'lat', 'long', 'Elevation'], dtype='object')


(88705, 8)

# ___Scaling (10%)___

In [10]:
# *****************************
# HIGHLY IMPORTANT
# *****************************

# Sample data
print("Original Data Stats: \n")
print(data.describe())

print('\n--------\n')

print("New Sample Data Stats: \n")
# all_data['year'].fillna(0).astype(int)
# all_data['mass'].fillna(0).astype(int)


data = data.sample(frac=0.1)  # 10% sample set
print(data.describe())


Original Data Stats: 

                 id          mass         year          lat         long  \
count   8870.000000  4.528000e+03  8845.000000  8129.000000  8129.000000   
mean   37088.161443  1.052552e+04  1999.494291     2.419902    39.815265   
std    23712.552743  3.578561e+05    18.989401    52.968619    73.800775   
min       12.000000  0.000000e+00  1632.000000   -87.366670  -165.433330   
25%    16323.250000  6.900000e+00  1998.000000   -71.500000     5.611010   
50%    34040.500000  3.070000e+01  2002.000000    28.759444    21.758056   
75%    57024.500000  1.913500e+02  2008.000000    47.381944    75.316670   
max    80667.000000  2.200000e+07  2020.000000    74.683333   178.050000   

         Elevation  
count  4330.000000  
mean    345.282910  
std     564.852553  
min       0.000000  
25%       0.000000  
50%      80.000000  
75%     447.500000  
max    9999.000000  

--------

New Sample Data Stats: 

                 id          mass         year         lat        l

In [12]:
all_data = pd.DataFrame(data=data)
print(all_data)
# all_data.columns = data.feature_names
# data['year'] = data.target

# print(data.head())
all_data.columns

            id class     mass   fall    year        lat        long  Elevation
34395  21239.0    L6    12.40  Found  1997.0 -84.000000  168.000000        NaN
48330  62598.0   NaN      NaN    NaN  2009.0  53.175000   11.000000       80.0
41244  27885.0    H4    26.86  Found  1979.0 -71.500000   35.666670        NaN
73115  30041.0   NaN      NaN    NaN  2003.0  10.473333  -66.823333      800.0
57656  52849.0   NaN      NaN    NaN  1999.0  39.966667    0.016667       25.0
...        ...   ...      ...    ...     ...        ...         ...        ...
82166  13327.0   NaN      NaN    NaN  2004.0  18.541667   73.571667        0.0
38078  24064.0    H5  2300.00  Found  1981.0  34.666670 -102.000000        NaN
61495  71349.0   NaN      NaN    NaN  2015.0  53.116667   -2.066667        0.0
41900  28603.0    H5    38.19  Found  1979.0 -71.500000   35.666670        NaN
40166  26286.0   LL6     0.85  Found  1979.0 -71.500000   35.666670        NaN

[887 rows x 8 columns]


Index(['id', 'class', 'mass', 'fall', 'year', 'lat', 'long', 'Elevation'], dtype='object')